In [46]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125008")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125008
Azure region: southcentralus
Subscription id: 13fe1ad9-91fc-4300-9ca0-27ebe69e0476
Resource group: aml-quickstarts-125008


In [47]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Nane of the CPU cluster
amlcompute_cluster_name = "my-compute"

# Verify thet cluster does not exist
try:
    aml_compute = ComputeTarget(workspace= ws, name=amlcompute_cluster_name)
    print('Found existing cluster, will use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(wm_size = 'STANDARD_D2_V2', max_nodes=4)
    aml_compute.wait_for_completion(show_output=True)

Found existing cluster, will use it!


In [48]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({
    '--C': uniform(0.5, 1.5),
    '--max_iter': choice(50, 150)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.07)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target='my-compute',
entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=10,
max_concurrent_runs=2)
print('HyperDriveConfig has been created!')

HyperDriveConfig has been created!


In [52]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

my_run = exp.submit(config=hyperdrive_config)
RunDetails(my_run).show()
my_run.wait_for_completion(show_output=True)
assert(my_run.get_status()=="Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_28df2737-257e-4852-afb9-65c9ec478bc6
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_28df2737-257e-4852-afb9-65c9ec478bc6?wsid=/subscriptions/13fe1ad9-91fc-4300-9ca0-27ebe69e0476/resourcegroups/aml-quickstarts-125008/workspaces/quick-starts-ws-125008

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-07T11:10:39.288186][API][INFO]Experiment created<END>\n""<START>[2020-11-07T11:10:39.858497][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-11-07T11:10:40.030369][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-07T11:10:40.7187507Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_28df2737-257e-4852-afb9-65c9ec478bc6
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_28df2737-257e-4852-afb9-65c9ec478bc6?wsid=/subscriptions/13fe1

In [55]:
import joblib
# Get your best run and save the model from that run.

best_run = my_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
run_definition = parameter_values.get("runDefinition")

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Arguments for best run:', run_definition.get("arguments"))
print()
print(best_run.get_file_names())

# Register model
model = best_run.register_model(model_name="my_best_run", model_path="outputs/model.joblib")

# Check model
from azureml.core import Model
for model in Model.list(ws):
    print(model.name)

Best Run Id:  HD_28df2737-257e-4852-afb9-65c9ec478bc6_3
Accuracy: 0.9119932022335518
Arguments for best run: ['--C', '0.9246805964707864', '--max_iter', '150']

['azureml-logs/55_azureml-execution-tvmps_7b85898d2dec7a5dae524be87f62e791940496ec512b3eb4295b31321a27e16f_d.txt', 'azureml-logs/65_job_prep-tvmps_7b85898d2dec7a5dae524be87f62e791940496ec512b3eb4295b31321a27e16f_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_7b85898d2dec7a5dae524be87f62e791940496ec512b3eb4295b31321a27e16f_d.txt', 'logs/azureml/100_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_160d647f-3539-4d86-b148-94bb4266bfc0.jsonl', 'logs/azureml/dataprep/python_span_l_160d647f-3539-4d86-b148-94bb4266bfc0.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


my_best_run


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###